In [9]:
import time
import networkx as nx

import mccSearch
import variables
import iomethods

userVariables = variables.define_user_variables(False)
userVariables.DIRS['mainDirStr'] = "/path/to/store/results"
userVariables.DIRS['TRMMdirName'] = "/path/to/TRMM/data"
userVariables.DIRS['CEoriDirName'] = "/path/to/MERG/data"
# set latitudes and longitudes
userVariables.LATMIN = '5.0'
userVariables.LATMAX = '19.0'
userVariables.LONMIN = '-5.0'
userVariables.LONMAX = '9.0'
# set start and end times
userVariables.startDateTime = "200609110000"
userVariables.endDateTime = "200609121200"

In [10]:
#read data
graphVariables = iomethods.read_vars(userVariables)

Directory exists already!!!


In [11]:
#create test output file
unittestFile = open(userVariables.DIRS['mainDirStr']+'/unittestResults.txt','wb')
unittestFile.write("\n Timing results for "+userVariables.startDateTime+" to "+userVariables.endDateTime)

In [12]:
# ingest data from files
print "Start the timer"
# time how long it takes to complete reading in the data
print "\n -------------- Read MERG Data ----------"
print "\n Start the timer for the data ingest process"
readMergStart = time.time()
mergImgs, timeList, LAT, LON,userVariables = iomethods.read_data('ch4','latitude','longitude', userVariables)
readMergEnd = time.time()
timeTaken = readMergEnd - readMergStart
print "\n End the timer for the data ingest process"
print "\n Total time to complete data ingest is %g seconds"%(readMergEnd - readMergStart)
unittestFile.write("\n 1. Total time to complete data ingest is %g seconds"%(readMergEnd - readMergStart))

Start the timer

 -------------- Read MERG Data ----------

 Start the timer for the data ingest process

 End the timer for the data ingest process

 Total time to complete data ingest is 1.75667 seconds


In [13]:
#find cloud elements
CEGraph = nx.DiGraph()
prunedGraph = nx.DiGraph()
MCCList = []
MCSList = []

print "\n -------------- TESTING findCloudElements ----------"
print "\n Start the timer for findCloudElements process"
findCEsStart = time.time()
print "\n Using both MERG and TRMM simultaneously "
CEGraph = mccSearch.find_cloud_elements(mergImgs,timeList, LAT, LON, userVariables, graphVariables, userVariables.DIRS['TRMMdirName'])
findCEsEnd = time.time()
print "\n Number of cloud elements found is: ", CEGraph.number_of_nodes()
print "\n End the timer for findCloudElements process"
print "\n Total time to complete finding cloud elements is %g seconds"%(findCEsEnd - findCEsStart)
timeTaken +=  findCEsEnd - findCEsStart
unittestFile.write("\n 2. Total time to complete finding cloud elements is %g seconds"%(findCEsEnd - findCEsStart))
print ("-"*80)


 -------------- TESTING findCloudElements ----------

 Start the timer for findCloudElements process

 Using both MERG and TRMM simultaneously 

 Number of cloud elements found is:  152

 End the timer for findCloudElements process

 Total time to complete finding cloud elements is 32.855 seconds
--------------------------------------------------------------------------------


In [14]:
#find cloud clusters
print "\n -------------- TESTING findCloudClusters ----------"
print "\n Start the timer for findCloudClusters process"
findCloudClustersStart = time.time()
prunedGraph = mccSearch.find_cloud_clusters(CEGraph,userVariables,graphVariables)
print "The number of nodes in the prunedGraph is: ", prunedGraph.number_of_nodes()
findCloudClustersEnd = time.time()
timeTaken += findCloudClustersEnd - findCloudClustersStart
print "\n End the timer for the findCloudClusters process"
print "\n Total time to complete finding the cloud clusters is %g seconds"%(findCloudClustersEnd - findCloudClustersStart)
unittestFile.write("\n 3. Total time to complete finding the cloud clusters is %g seconds"%(findCloudClustersEnd - findCloudClustersStart))
print ("-"*80)


 -------------- TESTING findCloudClusters ----------

 Start the timer for findCloudClusters process
pruned graph
number of nodes are:  127
number of edges are:  119
********************************************************************************
The number of nodes in the prunedGraph is:  127

 End the timer for the findCloudClusters process

 Total time to complete finding the cloud clusters is 0.0336909 seconds
--------------------------------------------------------------------------------


In [15]:
#find MCC
print "\n -------------- TESTING findMCCs ----------"
print "\n Start the timer for the findMCCs process"
findMCCStart = time.time()
MCCList,MCSList = mccSearch.find_MCC(prunedGraph,userVariables,graphVariables)
print "\n MCC List has been acquired ", len(MCCList)
print "\n MCS List has been acquired ", len(MCSList)
findMCCEnd = time.time()
timeTaken += findMCCEnd - findMCCStart
print "\n End the timer for the findMCCs process"
print "\n Total time to complete finding the MCCs is %g seconds"%(findMCCEnd - findMCCStart)
unittestFile.write("\n 4. Total time to complete finding the MCCs is %g seconds"%(findMCCEnd - findMCCStart))
print ("-"*80)
print ("*"*80)
print "\n The entire evaluation took %g seconds to complete" %(timeTaken)
unittestFile.write("\n The entire evaluation took %g seconds to complete" %(timeTaken))
unittestFile.write("\n ----------------------------------------------------------------")
unittestFile.write("\n Number of cloud elements found is: %d"%CEGraph.number_of_nodes())
unittestFile.write("\n Number of edges (with the cloud elements) found is: %d"%CEGraph.number_of_edges())
unittestFile.write("\n The number of nodes in the prunedGraph is: %d" %prunedGraph.number_of_nodes())
unittestFile.write("\n The number of edges (with nodes) in the prunedGraph is: %d" %prunedGraph.number_of_edges())
unittestFile.write("\n MCC List has been acquired %d" %len(MCCList))
unittestFile.write("\n MCS List has been acquired %d" %len(MCSList))


 -------------- TESTING findMCCs ----------

 Start the timer for the findMCCs process

 MCC List has been acquired  0

 MCS List has been acquired  5

 End the timer for the findMCCs process

 Total time to complete finding the MCCs is 12.4745 seconds
--------------------------------------------------------------------------------
********************************************************************************

 The entire evaluation took 47.1198 seconds to complete


In [16]:
#close test output file
unittestFile.close()